# 01 - AI Platform 教程

本教程完整演示 AI Platform 的端到端流程：从数据入库、模型训练到推理服务。

## 架构回顾

AI Platform 由两个微服务组成：

| 服务 | 端口 | 职责 |
|------|------|------|
| **Orchestrator** | 8000 | 用户唯一入口，任务编排、代理查询、推理服务 |
| **Executor** | 8001 | 数据湖存储（Lance）+ 脚本执行器（Daft） |

两者通过 **Lance 格式**作为契约层连接。用户只与 Orchestrator 交互，不直接接触 Executor。

```
用户 --> Orchestrator (8000) --> Executor (8001) --> Lance 数据湖
```

## 学习目标

- 理解 Orchestrator + Executor 微服务架构
- 通过 HTTP API 完成 MNIST 数据入库、CNN 训练、推理
- 观察 Lance 数据湖中的数据集和模型
- 可视化推理结果

## 前置条件

```bash
pip install -r requirements.txt
```

## 1. 启动服务

用 `subprocess` 在后台启动 Executor 和 Orchestrator 两个 uvicorn 进程。

启动顺序：先 Executor（8001），再 Orchestrator（8000），因为 Orchestrator 依赖 Executor。

In [ ]:
import subprocess
import time

import httpx

# 启动 Executor（数据湖 + 脚本执行器）
executor_proc = subprocess.Popen(
    ["uvicorn", "executor.app:app", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
print(f"Executor 已启动 (PID: {executor_proc.pid})")

# 启动 Orchestrator（用户入口）
orchestrator_proc = subprocess.Popen(
    ["uvicorn", "orchestrator.app:app", "--port", "8000"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
print(f"Orchestrator 已启动 (PID: {orchestrator_proc.pid})")

# 等待服务就绪
time.sleep(3)

# 定义 API 基地址（后续所有请求都发到 Orchestrator）
BASE_URL = "http://localhost:8000/api/v1"

验证服务是否正常运行：

In [ ]:
# 检查 Orchestrator 是否能连接到 Executor
resp = httpx.get(f"{BASE_URL}/datasets")
print(f"状态码: {resp.status_code}")
print(f"数据集列表: {resp.json()}")

## 2. 数据入库（Ingestion）

提交一个 `type=ingestion` 任务，让平台执行我们的清洗脚本 `mnist_clean.py`。

脚本会：
1. 自动下载 MNIST 数据集（60000 训练 + 10000 测试）
2. 将 28x28 图像展平为 784 维向量
3. 归一化像素值到 [0, 1]
4. 写入 Lance 格式

**注意**：首次运行需要下载 MNIST 数据（约 11MB），请耐心等待。

In [ ]:
# 提交数据入库任务
resp = httpx.post(f"{BASE_URL}/tasks", json={
    "type": "ingestion",
    "name": "mnist_ingestion",
    "input": "download",                                    # 自动下载 MNIST
    "script": "scripts/pipelines/mnist_clean.py",            # 清洗脚本路径
    "params": {"normalize": True},                           # 归一化像素值
    "output": "lance_storage/datasets/mnist_clean.lance",    # 输出到数据湖
})

ingestion_task = resp.json()
print(f"任务 ID: {ingestion_task['id']}")
print(f"状态: {ingestion_task['status']}")

任务在后台异步执行。我们通过轮询 `GET /tasks/{id}` 等待完成：

In [ ]:
# 轮询等待任务完成
task_id = ingestion_task["id"]
while True:
    resp = httpx.get(f"{BASE_URL}/tasks/{task_id}")
    task = resp.json()
    status = task["status"]
    if status in ("completed", "failed"):
        break
    print(f"状态: {status}，等待中...")
    time.sleep(5)

print(f"\n最终状态: {status}")
if status == "completed":
    print(f"结果: {task['result']}")
else:
    print(f"错误: {task.get('error')}")

## 3. 查看数据集

数据入库完成后，可以通过 `GET /datasets` 查看数据湖中的数据集。

注意响应中的 `schema` 字段——这就是 Lance 文件的列定义：
- `image`: 784 维浮点数组（展平后的 28x28 像素）
- `label`: 0-9 的整数标签
- `split`: "train" 或 "test"

In [ ]:
# 列出所有数据集
resp = httpx.get(f"{BASE_URL}/datasets")
datasets = resp.json()

for ds in datasets:
    print(f"数据集: {ds['id']}")
    print(f"  路径: {ds['path']}")
    print(f"  行数: {ds['num_rows']}")
    print(f"  Schema:")
    for col, dtype in ds["schema"].items():
        print(f"    {col}: {dtype}")

也可以查看单个数据集的详情：

In [ ]:
# 查看 mnist_clean 数据集详情
resp = httpx.get(f"{BASE_URL}/datasets/mnist_clean")
print(resp.json())

## 4. 模型训练（Training）

提交一个 `type=training` 任务，训练一个简单的 CNN 分类器。

训练脚本 `mnist_cnn.py` 会：
1. 从 Lance 数据湖读取训练数据
2. 构建 PyTorch DataLoader
3. 训练 CNN（两层卷积 + 两层全连接）
4. 评估测试集准确率
5. 将模型权重 + 指标保存回 Lance

我们用 3 个 epoch 做演示，CPU 上大约需要 1-2 分钟。

In [ ]:
# 提交训练任务
resp = httpx.post(f"{BASE_URL}/tasks", json={
    "type": "training",
    "name": "mnist_cnn_v1",
    "input": "lance_storage/datasets/mnist_clean.lance",     # 从数据湖读取
    "script": "scripts/training/mnist_cnn.py",               # 训练脚本
    "params": {
        "epochs": 3,
        "learning_rate": 0.001,
        "batch_size": 64,
        "device": "cpu",
    },
    "output": "lance_storage/models/mnist_cnn_v1.lance",     # 模型保存到数据湖
})

training_task = resp.json()
print(f"任务 ID: {training_task['id']}")
print(f"状态: {training_task['status']}")

In [ ]:
# 轮询等待训练完成
task_id = training_task["id"]
while True:
    resp = httpx.get(f"{BASE_URL}/tasks/{task_id}")
    task = resp.json()
    status = task["status"]
    if status in ("completed", "failed"):
        break
    print(f"状态: {status}，训练中...")
    time.sleep(10)

print(f"\n最终状态: {status}")
if status == "completed":
    result = task["result"]
    print(f"准确率: {result['accuracy']:.2%}")
    print(f"测试损失: {result['test_loss']:.4f}")
else:
    print(f"错误: {task.get('error')}")

## 5. 查看模型

训练完成后，模型权重和元数据都保存在 Lance 数据湖中。

Lance 表的 schema：
- `weights`: Binary — PyTorch state_dict 的序列化字节
- `params`: String — 超参数 JSON
- `metrics`: String — 训练指标 JSON
- `created_at`: String — 创建时间

In [ ]:
# 列出所有模型
resp = httpx.get(f"{BASE_URL}/models")
models = resp.json()

for m in models:
    print(f"模型: {m['id']}")
    print(f"  路径: {m['path']}")
    print(f"  Schema:")
    for col, dtype in m["schema"].items():
        print(f"    {col}: {dtype}")

## 6. 启动推理服务（Inference）

提交一个 `type=inference` 任务，Orchestrator 会：
1. 通过 Executor API 获取模型的 Lance 文件路径
2. 用 Daft 读取 Lance 中的模型权重
3. 用 PyTorch 加载权重到 CNN 模型
4. 模型进入 eval 模式，准备接收 predict 请求

与批处理任务不同，推理任务是**常驻服务**——一直保持 `running` 状态直到手动 `cancel`。

In [ ]:
# 启动推理服务
resp = httpx.post(f"{BASE_URL}/tasks", json={
    "type": "inference",
    "name": "mnist_predictor",
    "model": "mnist_cnn_v1",    # 从数据湖加载这个模型
    "device": "cpu",
    "port": 8080,
})

inference_task = resp.json()
print(f"任务 ID: {inference_task['id']}")
print(f"状态: {inference_task['status']}")
print(f"端点: {inference_task.get('endpoint', 'N/A')}")

## 7. 调用推理

推理服务就绪后，通过 `POST /tasks/{id}/predict` 发送图像数据。

请求体是 784 维浮点数组（28x28 归一化像素值）。

我们从数据湖中取一张真实的测试图片来试试：

In [ ]:
import daft

# 从数据湖读取测试集中的前 5 张图片
df = daft.read_lance("lance_storage/datasets/mnist_clean.lance")
pdf = df.to_pandas()
test_samples = pdf[pdf["split"] == "test"].head(5)

print(f"取出 {len(test_samples)} 张测试图片")
print(f"真实标签: {test_samples['label'].tolist()}")

In [ ]:
# 对每张图片调用推理 API
inference_id = inference_task["id"]
predictions = []

for i, row in test_samples.iterrows():
    image_data = [float(x) for x in row["image"]]  # 转为 Python float 列表
    resp = httpx.post(
        f"{BASE_URL}/tasks/{inference_id}/predict",
        json={"image": image_data},
    )
    result = resp.json()
    predictions.append(result)
    print(f"真实: {row['label']}, 预测: {result['prediction']}, 置信度: {result['confidence']:.4f}")

### 可视化预测结果

将 784 维向量还原为 28x28 图像，展示预测结果：

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for idx, (ax, (_, row)) in enumerate(zip(axes, test_samples.iterrows())):
    # 将 784 维向量还原为 28x28 图像
    image = np.array(row["image"]).reshape(28, 28)
    ax.imshow(image, cmap="gray")

    pred = predictions[idx]
    true_label = row["label"]
    pred_label = pred["prediction"]
    confidence = pred["confidence"]

    # 预测正确显示绿色，错误显示红色
    color = "green" if pred_label == true_label else "red"
    ax.set_title(f"真实: {true_label}\n预测: {pred_label} ({confidence:.1%})", color=color)
    ax.axis("off")

plt.suptitle("MNIST 推理结果", fontsize=14)
plt.tight_layout()
plt.show()

### 概率分布

查看第一张图片的 10 类概率分布：

In [ ]:
# 第一张图片的概率分布
probs = predictions[0]["probabilities"]

fig, ax = plt.subplots(figsize=(8, 4))
bars = ax.bar(range(10), probs)

# 高亮预测类别
pred_idx = predictions[0]["prediction"]
bars[pred_idx].set_color("green")

ax.set_xlabel("数字类别")
ax.set_ylabel("概率")
ax.set_title(f"预测概率分布（真实标签: {test_samples.iloc[0]['label']}）")
ax.set_xticks(range(10))
plt.tight_layout()
plt.show()

## 8. 查看所有任务

回顾我们创建的所有任务：

In [ ]:
# 列出所有任务
resp = httpx.get(f"{BASE_URL}/tasks")
tasks = resp.json()

for t in tasks:
    print(f"{t['id']}  type={t['type']:<10}  status={t['status']:<10}  name={t['name']}")

也可以按类型过滤：

In [ ]:
# 只看推理任务
resp = httpx.get(f"{BASE_URL}/tasks", params={"type": "inference"})
print(resp.json())

## 9. 停止推理服务

推理任务是常驻服务，需要手动取消。取消后模型会从内存中卸载。

In [ ]:
# 停止推理服务
resp = httpx.post(f"{BASE_URL}/tasks/{inference_task['id']}/cancel")
print(f"推理服务已停止: {resp.json()}")

## 10. 清理

停止 Executor 和 Orchestrator 进程。

In [ ]:
# 停止服务
orchestrator_proc.terminate()
executor_proc.terminate()
orchestrator_proc.wait()
executor_proc.wait()
print("所有服务已停止")

## 总结

本教程完整演示了 AI Platform 的三种任务类型：

| 步骤 | API | 任务类型 | 说明 |
|------|-----|---------|------|
| 数据入库 | `POST /tasks` | ingestion | 下载 MNIST，归一化，写入 Lance |
| 查看数据集 | `GET /datasets` | — | 查看 schema 和行数 |
| 模型训练 | `POST /tasks` | training | CNN 训练，权重保存到 Lance |
| 查看模型 | `GET /models` | — | 查看模型元数据 |
| 启动推理 | `POST /tasks` | inference | 加载模型到内存 |
| 调用推理 | `POST /tasks/{id}/predict` | — | 发送图像，获取预测 |
| 停止推理 | `POST /tasks/{id}/cancel` | — | 卸载模型 |

### 关键设计点

- **统一任务 API**：三种任务共用 `/tasks` 端点，通过 `type` 区分
- **Lance 作为契约层**：数据集和模型都存储在 Lance 格式中，两个服务通过共享存储交换数据
- **用户脚本模式**：平台不绑定特定数据集或模型，用户提供自己的清洗/训练脚本
- **微服务解耦**：Orchestrator 不直接操作数据，通过 Executor API 代理

### 进阶方向

- Level 2: 引入 Ray，支持并发任务和 Ray Serve 推理
- Level 3: Ray on K8s + S3 共享存储，多机多任务
- 详见 [design.md](./design.md) 中的部署级别设计